Sources:
https://platform.openai.com/docs/guides/fine-tuning

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tiktoken

In [22]:
df = pd.read_excel('/home/soham/Downloads/Training_data.xlsx') 

A prompt separator string needs to be added at the end of the prompt both while fine tuning and sending requests to the model. Else, the outputted completions would most likely be random, instead of our desired output intents. This separator should NOT be present in the utterance at all.

In [23]:
prompt_separator = "\n\nIntent:\n\n"

Completions should start with a whitespace since OpenAI's tokenization tokenizes most words with a preceding whitespace.

For classification tasks, OpenAI recommends us to choose classes which map to a single token and set max_tokens=1. This is also a must when we want the model to output classification metrics. But some classes map to multiple tokens. <br>
Usually, we set a stop sequence at the end of the completion instead. But this won't work if we want to calculate classification metrics. So we convert the intent to strings of one token.

In [24]:
all_intents = df['Intent'].sort_values().unique()

In [25]:
enc = tiktoken.encoding_for_model('ada')

In [30]:
intent_completion_mapping = {all_intents[i]: enc.decode([i]) for i in range(len(all_intents))}
completion_intent_mapping = {enc.decode([i]): all_intents[i] for i in range(len(all_intents))}

In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [8]:
train_df = train_df[train_df['Intent'].isin(test_df['Intent'])]

In [9]:
test_df = test_df[test_df['Intent'].isin(train_df['Intent'])]

In [10]:
train_list = []
val_list = []

for idx, row in train_df.iterrows():
    train_list.append({'prompt': f'{row.Text}{prompt_separator}', 
                      'completion': intent_completion_mapping[row.Intent]})
for idx, row in test_df.iterrows():    
    val_list.append({'prompt': f'{row.Text}{prompt_separator}', 
                      'completion': intent_completion_mapping[row.Intent]})

In [11]:
len(train_list), len(val_list)

(3790, 950)

In [12]:
import jsonlines

In [13]:
with jsonlines.open('training_data_balic.jsonl', mode='w') as writer:
    # Write each data item as a separate line
    for item in train_list:
        writer.write(item)

In [14]:
with jsonlines.open('validation_data_balic.jsonl', mode='w') as writer:
    # Write each data item as a separate line
    for item in val_list:
        writer.write(item)

In [15]:
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

True

In [16]:
!openai api fine_tunes.create \
  -t  training_data_balic.jsonl\
  -v validation_data_balic.jsonl \
  -m ada \
  --compute_classification_metrics \
  --classification_n_classes 89


Upload progress: 100%|███████████████████████| 477k/477k [00:00<00:00, 968Mit/s]
Uploaded file from training_data_balic.jsonl: file-V08rPQtkfYsvb9RtqyNwXDs7
Upload progress: 100%|███████████████████████| 116k/116k [00:00<00:00, 157Mit/s]
Uploaded file from validation_data_balic.jsonl: file-znM0WR7lA6h0QRPuFK2AdQFX
Error: Billing hard limit has been reached (HTTP status code: 400)


In [13]:
!openai api fine_tunes.follow -i ft-CxcpkqXbEHbZBqZBvxk2fT43

[2023-08-18 16:33:41] Created fine-tune: ft-CxcpkqXbEHbZBqZBvxk2fT43
[2023-08-18 16:33:52] Fine-tune costs $0.18
[2023-08-18 16:33:53] Fine-tune enqueued. Queue number: 0
[2023-08-18 16:33:55] Fine-tune started
[2023-08-18 16:40:34] Completed epoch 1/4
[2023-08-18 16:54:01] Completed epoch 3/4
[2023-08-18 17:01:25] Uploaded model: ada:ft-personal-2023-08-18-11-31-25
[2023-08-18 17:01:26] Uploaded result file: file-I3e93KLmLAoDdHpA2OuEFMR2
[2023-08-18 17:01:26] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-personal-2023-08-18-11-31-25 -p <YOUR_PROMPT>


In [14]:
!openai api fine_tunes.results -i ft-CxcpkqXbEHbZBqZBvxk2fT43

step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/weighted_f1_score
1,164,4,0.47268741225143457,0.0,0.0,0.11909145454926864,0.0,0.0,,
2,296,8,0.517779551471881,0.0,0.0,,,,,
3,492,12,0.29366986377017684,0.0,0.0,,,,,
4,720,16,0.19366064928676807,0.0,0.0,,,,,
5,1012,20,0.13630273685573935,0.0,0.0,,,,,
6,1208,24,0.1463732886793635,0.0,0.0,,,,,
7,1436,28,0.1871233617945213,0.0,0.0,,,,,
8,1600,32,0.23126287840213627,0.0,0.0,,,,,
9,1764,36,0.20588405121339023,0.0,0.0,0.17944531372969508,0.0,0.0,,
10,2056,40,0.1056876451876532,0.0,0.0,,,,,
11,2508,44,0.057573390506939694,0.0,0.0,,,,,
12,2640,48,0.17188160304678604,0.0,0.0,,,,,
13,2804,52,0.1899889072645456,0.0,0.0,,,,,
14,3224,56,0.07122963718755133,0.0,0.0,,,,,
15,3324,60,0.276668827731628,0.0,0.0,,,,,
16,3488,64,0.19362371105444615,0.0,0.0,,,,,
17,3652,68,0.1767225561104715,0.0,0.0,

In [35]:
import openai
response = openai.Completion.create(
    model="ada:ft-personal-2023-08-18-11-31-25",
    prompt=f"Citrix configuration. User Issue:- Citrix configuration.User Name{prompt_separator}",
    max_tokens=1
)
response = response['choices'][0]['text']
print(completion_intent_mapping[response])

Citrix_Issue_Infra_Inc
